In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.table import Table



In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import WMAP9 as cosmo

# LET'S START WITH THE MASTER CATALOG FROM GOODS-S

In [ ]:
master_cat = pd.read_fwf("data/MASTERCAT_v3.0.dat", sep=" ", skiprows=62, header=None)
master_cat.columns = ["id", "location", "location2", "RA", "DEC", "ZSPEC", "Quality"]
#add a column to keep track of which cat the line in the DF comes from
master_cat['Source'] = 'Master'

In [ ]:
plt.scatter(master_cat['RA'], master_cat['DEC'], s = 0.1, c = master_cat['ZSPEC'], vmax = 4)
plt.colorbar(label = 'Redshift')
plt.figure()
plt.hist(master_cat['ZSPEC'], bins=100);

In [ ]:
master_cat[master_cat['id']==18]["Quality"]

The first thing we notice is that while in the header it is written "3,4,5,6,7,10,11,13:   NO QF provided", all the GMASS quality flag (id=18) are missing. They can be retrieved from the web server at http://archive.eso.org/wdb/wdb/vo/goods_CDFS_master/form (local file wdb_query_13059_eso.csv)

In [ ]:
goods_web = pd.read_csv("data/wdb_query_13059_eso.csv", comment="#")
#Let's make sure that the entries are in the same order
gmass_master = master_cat[master_cat["id"]==18]
assert len(goods_web)==len(gmass_master)
assert np.all(np.array(goods_web["Spect. z"])==np.array(gmass_master["ZSPEC"]))

In [ ]:
#Reassign the quality flag to the GMASS. This is 0 or 1, 1 being a secure redshift
master_cat.loc[master_cat["id"]==18, "Quality"] = np.array(goods_web["Quality flag"])
master_cat.loc[master_cat["id"]==18, "Source"] = "Master+web"
master_cat[master_cat['id']==18]

Next we need to deal with the entries in master catalogue without quality flags, as we want to be able to select secure zspec. These are INDEF in the ascii file, but pandas ends up keeping only the letter F

In [ ]:
no_qf_mask = master_cat["Quality"]=="F"
#we can check that it is equivalent to
#indef_mask = np.logical_or(master_cat["id"]==3, master_cat["id"]==4)
#indef_mask = np.logical_or(indef_mask, master_cat["id"]==5)
#indef_mask = np.logical_or(indef_mask, master_cat["id"]==6)
#indef_mask = np.logical_or(indef_mask, master_cat["id"]==7)
#indef_mask = np.logical_or(indef_mask, master_cat["id"]==10)
#indef_mask = np.logical_or(indef_mask, master_cat["id"]==11)
#indef_mask = np.logical_or(indef_mask, master_cat["id"]==13)
master_indef = master_cat[no_qf_mask]


The first thing we can try to do is to search for these indefinite flag cases in the CANDELS redshift catalog for GOODS-S from MAST  HLSP server : https://archive.stsci.edu/hlsp/candels/goods-s-catalogs

In [ ]:
f =open('data/hlsp_candels_hst_wfc3_goodss_multi_v2_redshift-cat.txt')
#build the column names out of the header
names = f.readlines()[3:60]
names = [name[1:-2] for name in names]
names[:9] = [name[3:] for name in names[:9]]
names[10:] = [name[4:] for name in names[10:]]
f.close()
#now read the file as a DataFrame, and rename some columns to follow master conventions
candels_table = pd.read_csv('data/hlsp_candels_hst_wfc3_goodss_multi_v2_redshift-cat.txt', sep = ' ', \
                            skiprows=61, header=None, names=names)
candels_table.rename({'z_spec':'ZSPEC','RA (CANDELS.GOODSS.F160W.v1_1.photom.cat)':'RA',\
                      'DEC (CANDELS.GOODSS.F160W.v1_1.photom.cat)':'DEC'}, axis=1, inplace=True)
#remove entries without a zspec; if a zspec is present, it is from a robust measurement.
candels_table = candels_table[candels_table['ZSPEC']!=-1.]

In [ ]:
# Spatial cross match
cat1_c = SkyCoord(ra=np.array(master_indef['RA'])*u.degree, dec=np.array(master_indef['DEC'])*u.degree)
cat2_c = SkyCoord(ra=np.array(candels_table['RA'])*u.degree, dec=np.array(candels_table['DEC'])*u.degree)
# cat2_c = SkyCoord(ra=np.array(specz_table['RA'])*u.degree, dec=np.array(specz_table['DEC'])*u.degree)
idx , d2d , d3d = cat1_c.match_to_catalog_sky(cat2_c)

#list(idx)
mask = np.logical_and((d2d<2.78e-5*u.degree),
               abs(candels_table.iloc[idx]["ZSPEC"].values-master_indef["ZSPEC"].values)<=1.e-3)



In [ ]:
# Spatial cross match
cat1_c = SkyCoord(ra=np.array(master_indef['RA'])*u.degree, dec=np.array(master_indef['DEC'])*u.degree)
cat2_c = SkyCoord(ra=np.array(candels_table['RA'])*u.degree, dec=np.array(candels_table['DEC'])*u.degree)
# cat2_c = SkyCoord(ra=np.array(specz_table['RA'])*u.degree, dec=np.array(specz_table['DEC'])*u.degree)
idx , d2d , d3d = cat1_c.match_to_catalog_sky(cat2_c)


for i in range(len(idx)):
    #0.1" match requested
    if(d2d[i]<2.78e-5*u.degree):
        if(abs(master_indef.iloc[i]["ZSPEC"]-candels_table.iloc[idx[i]]["z_best"])<=1.e-3):
            #print(i, idx[i], d2d[i].degree, candels_table.iloc[idx[i]]["z_best_type"],candels_table.iloc[idx[i]]["z_best"],\
            #      master_indef.iloc[i]["ZSPEC"], candels_table.iloc[idx[i]]["RA"],candels_table.iloc[idx[i]]["DEC"],\
            #     master_indef.iloc[i]["RA"], master_indef.iloc[i]["DEC"])
            #master_cat[no_qf_mask, "Quality"][i] =  candels_table.iloc[idx[i]]["z_best_type"]
            #master_indef.iloc[i]["Source"] =  "candels"
            master_cat.loc[no_qf_mask, "Quality"].iloc[i] = candels_table.iloc[idx[i]]["z_best_type"]
            #master_cat.loc[no_qf_mask, "Source"].iloc[i] = "candels"
master_cat.loc[no_qf_mask, "Quality"]